<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>LLMs for Data Science</h1>
<h1>Generative AI</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from datetime import datetime
import json

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 
import sqlite3

import openai
from openai import OpenAI

import transformers
from transformers import pipeline
from transformers import set_seed
set_seed(42) # Set the seed to get reproducible results

import langchain
import langchain_openai
from langchain_openai import ChatOpenAI
import langchain_core
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

import os
import gzip

import tqdm as tq
from tqdm.notebook import tqdm

import watermark

%load_ext watermark
%matplotlib inline

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.13.3
IPython version      : 9.7.0

Compiler    : Clang 17.0.0 (clang-1700.0.13.3)
OS          : Darwin
Release     : 25.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 107b7c3665c838b850317aff195e9ae6ca6dcf42

openai          : 2.8.0
pandas          : 2.3.3
json            : 2.0.9
numpy           : 2.3.5
tqdm            : 4.67.1
langchain_openai: 1.0.3
matplotlib      : 3.10.7
transformers    : 4.57.1
langchain       : 1.0.7
sqlite3         : 2.6.0
watermark       : 2.5.0
langchain_core  : 1.0.5



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# OpenAI Basic Usage

The first step is generate API key on the OpenAI website and store it as the "OPENAI_API_KEY" variable in your local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key

Then we are ready to instantiate the client

In [4]:
client = OpenAI()

We start by getting a list of supported models.

In [5]:
client.models.list()

SyncPage[Model](data=[Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'), Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'), Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'), Model(id='gpt-5.1-codex-mini', created=1763007109, object='model', owned_by='system'), Model(id='gpt-5.1-chat-latest', created=1762547951, object='model', owned_by='system'), Model(id='gpt-5.1-2025-11-13', created=1762800353, object='model', owned_by='system'), Model(id='gpt-5.1', created=1762800673, object='model', owned_by='system'), Model(id='gpt-5.1-codex', created=1762988221, object='model', owned_by='system'), Model(id='davinci-002', created=1692634301, object='model', owned_by='system'), Model(id='babbage-002', created=1692634615, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='mode

In [6]:
model_list = json.loads(client.models.list().model_dump_json())["data"]

In total we have 104 models

In [7]:
len(model_list)

104

Along with some information about each model...

In [8]:
model_list[:3]

[{'id': 'gpt-4-0613',
  'created': 1686588896,
  'object': 'model',
  'owned_by': 'openai'},
 {'id': 'gpt-4',
  'created': 1687882411,
  'object': 'model',
  'owned_by': 'openai'},
 {'id': 'gpt-3.5-turbo',
  'created': 1677610602,
  'object': 'model',
  'owned_by': 'openai'}]

But let's just get a list of model names

In [9]:
print("\n".join(sorted([model["id"] for model in model_list])))

babbage-002
chatgpt-4o-latest
codex-mini-latest
computer-use-preview
computer-use-preview-2025-03-11
dall-e-2
dall-e-3
davinci-002
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-3.5-turbo-1106
gpt-3.5-turbo-16k
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
gpt-4
gpt-4-0125-preview
gpt-4-0613
gpt-4-1106-preview
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4-turbo-preview
gpt-4.1
gpt-4.1-2025-04-14
gpt-4.1-mini
gpt-4.1-mini-2025-04-14
gpt-4.1-nano
gpt-4.1-nano-2025-04-14
gpt-4o
gpt-4o-2024-05-13
gpt-4o-2024-08-06
gpt-4o-2024-11-20
gpt-4o-audio-preview
gpt-4o-audio-preview-2024-10-01
gpt-4o-audio-preview-2024-12-17
gpt-4o-audio-preview-2025-06-03
gpt-4o-mini
gpt-4o-mini-2024-07-18
gpt-4o-mini-audio-preview
gpt-4o-mini-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-mini-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-transcribe
gpt-4o-mini-tts
gpt-4o-realtime-preview
gpt-4o-realtime-preview-2024-10-01
gpt-4o-realtime-preview-2024-

## Basic Prompt

The recommended model for exploration is `gpt-4.1`, so we'll stick with it for now. The basic setup is relatively straightforward:

In [10]:
response = client.chat.completions.create(
  model="gpt-4.1",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

Which produces a response object

In [11]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

Which we can treat as a `named tuple`. The model answer can be found in the "message" dictionary inside the "choices" list

In [12]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Superman's primary weakness is **kryptonite**, a radioactive mineral from his home planet, Krypton. Exposure to kryptonite, especially green kryptonite, weakens Superman and can even be fatal with prolonged contact.\n\nIn addition to kryptonite, Superman has a few other weaknesses:\n- **Red sun radiation:** Superman's powers come from Earth's yellow sun. Under a red sun like Krypton's, he loses his powers.\n- **Magic:** Superman is vulnerable to magical attacks and spells, as his invulnerability does not protect against magic.\n- **Lead:** While not a weakness, Superman cannot see through lead with his x-ray vision.\n\nThese vulnerabilities have been explored and used against Superman in various comics, movies, and TV series.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))

In [13]:
print(response.choices[0].message.content)

Superman's primary weakness is **kryptonite**, a radioactive mineral from his home planet, Krypton. Exposure to kryptonite, especially green kryptonite, weakens Superman and can even be fatal with prolonged contact.

In addition to kryptonite, Superman has a few other weaknesses:
- **Red sun radiation:** Superman's powers come from Earth's yellow sun. Under a red sun like Krypton's, he loses his powers.
- **Magic:** Superman is vulnerable to magical attacks and spells, as his invulnerability does not protect against magic.
- **Lead:** While not a weakness, Superman cannot see through lead with his x-ray vision.

These vulnerabilities have been explored and used against Superman in various comics, movies, and TV series.


To request multiple answers, we must include the `n` parameter with the number of answers we want

In [14]:
%%time
response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {
            "role": "user", 
             "content": "What are the different kinds of Kryptonite?"
        },
    ],
    n=3
)

CPU times: user 4.34 ms, sys: 2.14 ms, total: 6.48 ms
Wall time: 31.7 s


And we can access each of the answers individually int he choices list

In [15]:
for output in response.choices:
    print("==========")
    print(output.message.role.title()) 
    print("==========")
    print(output.message.content)
    print("==========\n")

Assistant
Kryptonite is a fictional mineral from the **Superman** universe, known for its various forms, each with unique effects on Superman and sometimes others. Over the decades, comics and related media have introduced several kinds. Here are the **most notable types of Kryptonite**:

---

### 1. **Green Kryptonite**
- **Effects:** The classic form; weakens and can eventually kill Superman and other Kryptonians with prolonged exposure. Causes pain, cell degeneration, and loss of powers.
- **Appearances:** Most common in comics, TV, and movies.

---

### 2. **Red Kryptonite**
- **Effects:** Has unpredictable, often bizarre effects on Kryptonians. Can cause mutations, emotional changes, or temporary powers. Effects vary with each exposure.
- **Examples:** Turn Superman into a dragon, split him into good and evil versions, induce amnesia, etc.

---

### 3. **Gold Kryptonite**
- **Effects:** Permanently removes a Kryptonian’s powers by altering their ability to metabolize yellow solar 

In [16]:
response.usage

CompletionUsage(completion_tokens=1960, prompt_tokens=16, total_tokens=1976, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [17]:
print(response.choices[0].message.content)

Kryptonite is a fictional mineral from the **Superman** universe, known for its various forms, each with unique effects on Superman and sometimes others. Over the decades, comics and related media have introduced several kinds. Here are the **most notable types of Kryptonite**:

---

### 1. **Green Kryptonite**
- **Effects:** The classic form; weakens and can eventually kill Superman and other Kryptonians with prolonged exposure. Causes pain, cell degeneration, and loss of powers.
- **Appearances:** Most common in comics, TV, and movies.

---

### 2. **Red Kryptonite**
- **Effects:** Has unpredictable, often bizarre effects on Kryptonians. Can cause mutations, emotional changes, or temporary powers. Effects vary with each exposure.
- **Examples:** Turn Superman into a dragon, split him into good and evil versions, induce amnesia, etc.

---

### 3. **Gold Kryptonite**
- **Effects:** Permanently removes a Kryptonian’s powers by altering their ability to metabolize yellow solar energy.
- 

# HuggingFace Basic Usage

HuggingFace relies on pipelines that (mostly) leverage locally run models. All we have to do is specify which task we are interested in and the model we want to use

## Unmasking

Let's look at a simple case of using the base uncase BERT model to fill in masked data. We start by instantiating the pipeline (which will download the model the first time you run it)

In [18]:
unmasker = pipeline('fill-mask', model='bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


In [19]:
output=unmasker("Artificial Intelligence [MASK] take over the world.")
output

[{'score': 0.3182433843612671,
  'token': 2064,
  'token_str': 'can',
  'sequence': 'artificial intelligence can take over the world.'},
 {'score': 0.18299736082553864,
  'token': 2097,
  'token_str': 'will',
  'sequence': 'artificial intelligence will take over the world.'},
 {'score': 0.056001488119363785,
  'token': 2000,
  'token_str': 'to',
  'sequence': 'artificial intelligence to take over the world.'},
 {'score': 0.045194871723651886,
  'token': 2015,
  'token_str': '##s',
  'sequence': 'artificial intelligences take over the world.'},
 {'score': 0.045153431594371796,
  'token': 2052,
  'token_str': 'would',
  'sequence': 'artificial intelligence would take over the world.'}]

After which we can just call the pipeline directly

### Model Bias

As these models are trained on text written by a large number of people, they are also reflective of common biases that are present in society. Depending on our application we may or may not need to take this into account.

In [20]:
unmasker("The man worked as a [MASK].")

[{'score': 0.09747566282749176,
  'token': 10533,
  'token_str': 'carpenter',
  'sequence': 'the man worked as a carpenter.'},
 {'score': 0.05238331854343414,
  'token': 15610,
  'token_str': 'waiter',
  'sequence': 'the man worked as a waiter.'},
 {'score': 0.04962718114256859,
  'token': 13362,
  'token_str': 'barber',
  'sequence': 'the man worked as a barber.'},
 {'score': 0.0378861241042614,
  'token': 15893,
  'token_str': 'mechanic',
  'sequence': 'the man worked as a mechanic.'},
 {'score': 0.03768095001578331,
  'token': 18968,
  'token_str': 'salesman',
  'sequence': 'the man worked as a salesman.'}]

In [21]:
unmasker("The woman worked as a [MASK].")

[{'score': 0.21981610357761383,
  'token': 6821,
  'token_str': 'nurse',
  'sequence': 'the woman worked as a nurse.'},
 {'score': 0.15974204242229462,
  'token': 13877,
  'token_str': 'waitress',
  'sequence': 'the woman worked as a waitress.'},
 {'score': 0.11547350883483887,
  'token': 10850,
  'token_str': 'maid',
  'sequence': 'the woman worked as a maid.'},
 {'score': 0.037969063967466354,
  'token': 19215,
  'token_str': 'prostitute',
  'sequence': 'the woman worked as a prostitute.'},
 {'score': 0.030423954129219055,
  'token': 5660,
  'token_str': 'cook',
  'sequence': 'the woman worked as a cook.'}]

# LangChain

We instantiate the LangChain interface for OpenAI

In [22]:
model = ChatOpenAI(model="gpt-4o")

In [23]:
messages = [
    SystemMessage(content="What was Superman's weakness?"),
]

output = model.invoke(messages)
output

/Users/bgoncalves/My Drive/DataForScience/LLMsForDataScience/.venv/lib/python3.13/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


AIMessage(content="Superman's most well-known weakness is Kryptonite, a radioactive mineral from his home planet of Krypton. Exposure to Kryptonite weakens Superman, stripping him of his powers and making him vulnerable. Different varieties of Kryptonite have different effects, but the most common form, green Kryptonite, is the one that typically debilitates him. Additionally, Superman is vulnerable to magic and the effects of a red sun, conditions under which he loses his superhuman abilities.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 13, 'total_tokens': 105, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_74ab616da4', 'id': 'chatcmpl-CfRRI5fMzQlDhAJinVsFwOKoHSf5F', 'se

In [24]:
output.response_metadata["token_usage"]

{'completion_tokens': 92,
 'prompt_tokens': 13,
 'total_tokens': 105,
 'completion_tokens_details': {'accepted_prediction_tokens': 0,
  'audio_tokens': 0,
  'reasoning_tokens': 0,
  'rejected_prediction_tokens': 0},
 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}

In [25]:
parser = StrOutputParser()

In [26]:
result = model.invoke(messages)

In [27]:
parser.invoke(result)

"Superman's most well-known weakness is Kryptonite, a mineral from his home planet of Krypton. Exposure to Kryptonite weakens Superman significantly, and in the case of prolonged exposure, it can be deadly. There are different variations of Kryptonite, with green Kryptonite being the most common form that affects him. Other forms, like red, blue, or gold Kryptonite, have various effects ranging from unpredictable physical transformations to stripping him of his powers temporarily.\n\nIn addition to Kryptonite, Superman has vulnerabilities to magic and certain types of radiation, particularly red solar radiation, which can diminish his powers by mimicking the conditions of Krypton's red sun. Unlike Earth's yellow sun, which is the source of his powers, red solar radiation can render Superman effectively human in terms of strength and durability."

Let us create our first chain. Stages of the chain are conencted with the pipe '|' character

In [28]:
chain = model | parser

Now whenver we call __invoke()__ on the chain, it automatically runs all the steps

In [29]:
chain.invoke(messages)

"Superman's primary weakness is Kryptonite, a fictional mineral from his home planet of Krypton. Exposure to various forms of Kryptonite can have adverse effects on him, with the most common green Kryptonite causing him to lose his powers and become severely weakened, akin to a human being exposed to radiation poisoning. In addition to Kryptonite, Superman can be vulnerable to magic and certain forms of high-level advanced technology. His powers also rely on solar energy from Earth's yellow sun, so being deprived of sunlight can weaken him over time."

# Applications

## Text to Code

In [30]:
messages = [
        {
            "role": "system", 
            "content": """You are a grumpy but expert Python software engineer 
            thats interviewing for a job. Please be as concise with your answers as possible."""
        },
        {
            "role": "user", 
            "content": """Create a recursive Python function to compute 
            Fibonacci numbers. Don't provide any explanation, just the code"""
        },
  ]

In [31]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=messages,
    temperature=0,
    max_tokens=1024
)

Which produces the expected result

In [32]:
print(response.choices[0].message.content)

def fibonacci(n):
    if n <= 1:
       return n
    else:
       return(fibonacci(n-1) + fibonacci(n-2))


and works as expected

In [33]:
def fibonacci(n):
    if n <= 1:
       return n
    else:
       return(fibonacci(n-1) + fibonacci(n-2))

In [34]:
fibonacci(20)

6765

Let us define a utility function to make sequential queries easier

In [35]:
def chat(messages, prompt):
    messages.append({"role":"user", "content":prompt})
    
    response = client.chat.completions.create(
        model="gpt-4",
        messages=messages,
        temperature=0,
        max_tokens=1024
    )
    
    messages.append(response.choices[0].message)
    
    return messages[-1].content

## Adding comments

In [36]:
print(chat(messages, "Can you add comments to this function?"))

def fibonacci(n):
    # Base case: if n is 0 or 1, return n
    if n <= 1:
       return n
    # Recursive case: return the sum of the previous two Fibonacci numbers
    else:
       return(fibonacci(n-1) + fibonacci(n-2))


In [37]:
print(chat(messages, "What is the purpose of recursion in this piece of code?"))

Recursion in this code is used to break down the problem of calculating a Fibonacci number into smaller, simpler problems. It allows the function to call itself with different arguments, computing the smaller Fibonacci numbers first, then adding them together to get the desired result.


In [38]:
messages

[{'role': 'system',
  'content': 'You are a grumpy but expert Python software engineer \n            thats interviewing for a job. Please be as concise with your answers as possible.'},
 {'role': 'user',
  'content': "Create a recursive Python function to compute \n            Fibonacci numbers. Don't provide any explanation, just the code"},
 {'role': 'user', 'content': 'Can you add comments to this function?'},
 ChatCompletionMessage(content='def fibonacci(n):\n    # Base case: if n is 0 or 1, return n\n    if n <= 1:\n       return n\n    # Recursive case: return the sum of the previous two Fibonacci numbers\n    else:\n       return(fibonacci(n-1) + fibonacci(n-2))', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None),
 {'role': 'user',
  'content': 'What is the purpose of recursion in this piece of code?'},
 ChatCompletionMessage(content='Recursion in this code is used to break down the problem of calculating a Fibonacci number into sma

## Explaining Code

Let's use a relatively small python script

In [39]:
code_text = "".join(open("data/EpiModel.py").readlines())

In [40]:
print(code_text)

### −∗− mode : python ; −∗−
# @file EpiModel.py
# @author Bruno Goncalves
######################################################

import networkx as nx
import numpy as np
from numpy import linalg
from numpy import random
import scipy.integrate
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
tqdm.pandas()

class EpiModel(object):
    """Simple Epidemic Model Implementation
    
        Provides a way to implement and numerically integrate 
    """
    def __init__(self, compartments=None):
        self.transitions = nx.MultiDiGraph()
        self.seasonality = None
        
        if compartments is not None:
            self.transitions.add_nodes_from([comp for comp in compartments])
    
    def add_interaction(self, source, target, agent, rate):        
        self.transitions.add_edge(source, target, agent=agent, rate=rate)        
        
    def add_spontaneous(self, source, target, rate):
        self.transitions.add_edge(source, target, rate=rate)



In [41]:
%%time
print(chat(messages, "Please explain what this piece of code does: ```%s```" % code_text))

This code defines a class `EpiModel` for simulating and analyzing epidemic models. The class allows for the creation of a model with multiple compartments (like susceptible, infected, and recovered in an SIR model), and transitions between these compartments. 

The transitions can be interactions between compartments (like susceptible individuals becoming infected), spontaneous transitions (like infected individuals recovering), or vaccinations. 

The class also includes methods for simulating the epidemic over time, either stochastically or by numerical integration, and for plotting the results. 

The `R0` method calculates the basic reproduction number of the epidemic, which is the expected number of secondary infections produced by a single infected individual in a completely susceptible population.

The code at the end of the script creates an instance of the `EpiModel` class for an SIR model with vaccination, simulates it for a year, and plots the results.
CPU times: user 5.19 ms,

In [42]:
%%time
print(chat(messages, "Can you please add a doc string to each function and method? Please include information about each argument of the function"))

Sure, here are the docstrings for some of the methods in the `EpiModel` class:

```python
class EpiModel(object):
    """Simple Epidemic Model Implementation
    
    Provides a way to implement and numerically integrate 
    """
    def __init__(self, compartments=None):
        """
        Initialize the EpiModel object.
        
        Args:
            compartments (list): A list of compartments for the model.
        """
        # ...

    def add_interaction(self, source, target, agent, rate):        
        """
        Add an interaction transition to the model.
        
        Args:
            source (str): The source compartment.
            target (str): The target compartment.
            agent (str): The agent that causes the transition.
            rate (float): The rate of the transition.
        """
        # ...

    def add_spontaneous(self, source, target, rate):
        """
        Add a spontaneous transition to the model.
        
        Args:
            sour

## Interacting with a database

Let us open a small test database. This file was downloaded from https://github.com/chineseballer06/Statistical-Analysis-of-Northwind-Database/blob/master/Northwind_small.sqlite

In [43]:
con = sqlite3.connect("data/Northwind_small.sqlite")

In [44]:
messages = [
    {
        "role": "system", 
        "content": """You're a Database Administrator. 
                      Please generate SQL queries to answer the following questions. 
                      No comments are necessary."""
    },
    {
        "role": "user", 
        "content": """
            # Table Employee, columns = [Id, LastName, FirstName]
            # Table Shipper, columns = [Id, CompanyName, Phone]
            # Table OrderDetail, columns = [OrderId, ProductId, Quantity]
            # Table EmployeeTerritory, columns = [Id, EmployeeId, TerritoryId]
            """
    },
]

In [45]:
query_sql = chat(messages, "Generate a table with employee first name, last name and territory id")
print(query_sql)

SELECT Employee.FirstName, Employee.LastName, EmployeeTerritory.TerritoryId
FROM Employee
JOIN EmployeeTerritory ON Employee.Id = EmployeeTerritory.EmployeeId;


In [46]:
pd.read_sql(query_sql, con)

,FirstName,LastName,TerritoryId
0,Nancy,Davolio,06897
1,Nancy,Davolio,19713
2,Andrew,Fuller,01581
3,Andrew,Fuller,01730
4,Andrew,Fuller,01833
5,Andrew,Fuller,02116
6,Andrew,Fuller,02139
7,Andrew,Fuller,02184
8,Andrew,Fuller,40222
9,Janet,Leverling,30346


In [47]:
sql_query = chat(messages, "Compute how many employees work in each territory")
print(sql_query)

SELECT EmployeeTerritory.TerritoryId, COUNT(Employee.Id) as EmployeeCount
FROM Employee
JOIN EmployeeTerritory ON Employee.Id = EmployeeTerritory.EmployeeId
GROUP BY EmployeeTerritory.TerritoryId;


In [48]:
pd.read_sql(sql_query, con)

,TerritoryId,EmployeeCount
0,01581,1
1,01730,1
2,01833,1
3,02116,1
4,02139,1
5,02184,1
6,02903,1
7,03049,1
8,03801,1
9,06897,1


In [49]:
sql_query = chat(messages, "How many shippers do we work with?")
print(sql_query)

SELECT COUNT(*) FROM Shipper;


In [50]:
pd.read_sql(sql_query, con)

,COUNT(*)
0,3


In [51]:
messages

[{'role': 'system',
  'content': "You're a Database Administrator. \n                      Please generate SQL queries to answer the following questions. \n                      No comments are necessary."},
 {'role': 'user',
  'content': '\n            # Table Employee, columns = [Id, LastName, FirstName]\n            # Table Shipper, columns = [Id, CompanyName, Phone]\n            # Table OrderDetail, columns = [OrderId, ProductId, Quantity]\n            # Table EmployeeTerritory, columns = [Id, EmployeeId, TerritoryId]\n            '},
 {'role': 'user',
  'content': 'Generate a table with employee first name, last name and territory id'},
 ChatCompletionMessage(content='SELECT Employee.FirstName, Employee.LastName, EmployeeTerritory.TerritoryId\nFROM Employee\nJOIN EmployeeTerritory ON Employee.Id = EmployeeTerritory.EmployeeId;', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None),
 {'role': 'user',
  'content': 'Compute how many employe

<center>
     <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>